
# Subgraph Link Prediction Heuristics

This notebook computes link prediction heuristics for every prepared subgraph under `data/subgraphs`. For each subgraph we report the common neighbors count, Jaccard coefficient, Adamic–Adar index, and a network proximity z-score for all observed edges. Adjust the helper code if you want to look at non-edge pairs or persist the results to disk.


In [ ]:

from pathlib import Path
import math
import random

import networkx as nx
import numpy as np
import pandas as pd


In [ ]:

# Resolve project paths so the notebook works when launched from the repo root or the notebooks folder.
PROJECT_ROOT = Path.cwd().resolve()
if not (PROJECT_ROOT / 'data').exists():
    PROJECT_ROOT = PROJECT_ROOT.parent.resolve()

SUBGRAPH_ROOT = PROJECT_ROOT / 'data' / 'subgraphs'
if not SUBGRAPH_ROOT.exists():
    raise FileNotFoundError(f'Expected subgraphs under {SUBGRAPH_ROOT}')

subgraph_dirs = sorted(p for p in SUBGRAPH_ROOT.glob('*') if p.is_dir())
print(f"Found {len(subgraph_dirs)} subgraph(s):")
for path in subgraph_dirs:
    print(f" - {path.relative_to(PROJECT_ROOT)}")


In [ ]:

def load_subgraph(subgraph_path: Path):
    """Return node and edge tables for a subgraph directory."""
    nodes = pd.read_csv(subgraph_path / 'node.csv', sep='	', dtype={'node_index': str})
    edges = pd.read_csv(
        subgraph_path / 'edges.csv',
        dtype={'x_index': str, 'y_index': str, 'relation': str, 'display_relation': str},
    )
    return nodes, edges


def build_graph(nodes: pd.DataFrame, edges: pd.DataFrame) -> nx.Graph:
    """Construct an undirected graph from node and edge tables."""
    graph = nx.Graph()
    for row in nodes.itertuples(index=False):
        graph.add_node(row.node_index, node_type=row.node_type, node_name=row.node_name)

    for row in edges.itertuples(index=False):
        graph.add_edge(row.x_index, row.y_index, relation=row.relation)
    return graph


def compute_link_metrics(
    graph: nx.Graph,
    nodes: pd.DataFrame,
    edges: pd.DataFrame,
    *,
    proximity_samples: int = 200,
    seed: int = 42,
) -> pd.DataFrame:
    """Compute link prediction heuristics for every observed edge."""
    type_map = nodes.set_index('node_index')['node_type'].to_dict()
    name_map = nodes.set_index('node_index')['node_name'].to_dict()
    degree_map = dict(graph.degree())
    nodes_by_type = nodes.groupby('node_type')['node_index'].apply(list).to_dict()

    rng = random.Random(seed)
    shortest_path_cache: dict[str, dict[str, int]] = {}

    def shortest_path_length(u: str, v: str) -> float:
        if u not in shortest_path_cache:
            shortest_path_cache[u] = nx.single_source_shortest_path_length(graph, u)
        return shortest_path_cache[u].get(v, math.inf)

    baseline_cache: dict[tuple[str, str], tuple[float, float] | None] = {}

    def proximity_baseline(type_u: str, type_v: str):
        key = (type_u, type_v)
        if key in baseline_cache:
            return baseline_cache[key]

        candidates_u = nodes_by_type.get(type_u, [])
        candidates_v = nodes_by_type.get(type_v, [])
        if not candidates_u or not candidates_v:
            baseline_cache[key] = None
            baseline_cache[(type_v, type_u)] = None
            return None

        samples: list[float] = []
        seen_pairs: set[tuple[str, str]] = set()
        max_attempts = proximity_samples * 10
        attempts = 0

        while len(samples) < proximity_samples and attempts < max_attempts:
            attempts += 1
            u = rng.choice(candidates_u)
            v = rng.choice(candidates_v)
            if u == v:
                continue
            pair_key = (u, v) if type_u != type_v else tuple(sorted((u, v)))
            if pair_key in seen_pairs:
                continue
            seen_pairs.add(pair_key)

            dist = shortest_path_length(u, v)
            if math.isinf(dist):
                continue
            samples.append(dist)

        if len(samples) < 2:
            baseline_cache[key] = None
            baseline_cache[(type_v, type_u)] = None
            return None

        mean_dist = float(np.mean(samples))
        std_dist = float(np.std(samples, ddof=1)) if len(samples) > 1 else 0.0
        baseline_cache[key] = (mean_dist, std_dist)
        baseline_cache[(type_v, type_u)] = (mean_dist, std_dist)
        return baseline_cache[key]

    edge_pairs = [(row.x_index, row.y_index) for row in edges.itertuples(index=False)]

    jaccard_scores = {}
    for u, v, score in nx.jaccard_coefficient(graph, edge_pairs):
        jaccard_scores[(u, v)] = score
        jaccard_scores[(v, u)] = score

    adamic_scores = {}
    for u, v, score in nx.adamic_adar_index(graph, edge_pairs):
        adamic_scores[(u, v)] = score
        adamic_scores[(v, u)] = score

    records = []
    for row in edges.itertuples(index=False):
        u = row.x_index
        v = row.y_index
        source_type = type_map.get(u)
        target_type = type_map.get(v)

        cn_count = sum(1 for _ in nx.common_neighbors(graph, u, v))

        jaccard_score = jaccard_scores.get((u, v), float('nan'))
        adamic_score = adamic_scores.get((u, v), float('nan'))

        sp_length = shortest_path_length(u, v)
        if math.isinf(sp_length):
            proximity_z = None
            baseline_mean = None
            baseline_std = None
            sp_value = None
        else:
            baseline = proximity_baseline(source_type, target_type)
            if baseline is None:
                proximity_z = None
                baseline_mean = None
                baseline_std = None
            else:
                mean_dist, std_dist = baseline
                if std_dist > 0:
                    proximity_z = (sp_length - mean_dist) / std_dist
                else:
                    proximity_z = 0.0
                baseline_mean = mean_dist
                baseline_std = std_dist
            sp_value = sp_length

        records.append(
            {
                'source_index': u,
                'target_index': v,
                'source_type': source_type,
                'target_type': target_type,
                'source_name': name_map.get(u),
                'target_name': name_map.get(v),
                'relation': row.relation,
                'common_neighbors': cn_count,
                'jaccard_coefficient': jaccard_score,
                'adamic_adar_index': adamic_score,
                'shortest_path_length': sp_value,
                'network_proximity_z': proximity_z,
                'proximity_baseline_mean': baseline_mean,
                'proximity_baseline_std': baseline_std,
                'source_degree': degree_map.get(u),
                'target_degree': degree_map.get(v),
            }
        )

    return pd.DataFrame.from_records(records)


In [ ]:

results = {}
for subgraph_path in subgraph_dirs:
    nodes_df, edges_df = load_subgraph(subgraph_path)
    graph = build_graph(nodes_df, edges_df)
    metrics_df = compute_link_metrics(graph, nodes_df, edges_df)
    key = subgraph_path.name
    results[key] = {
        'graph': graph,
        'nodes': nodes_df,
        'edges': edges_df,
        'metrics': metrics_df,
    }
    print(f"Computed metrics for {key}: {len(metrics_df)} edge(s)")


In [ ]:

# Preview the first few rows for one subgraph.
example_key = next(iter(results))
results[example_key]['metrics'].head()


In [ ]:

# Optional: uncomment to export the metrics tables to CSV files.
# output_dir = PROJECT_ROOT / 'notebooks' / 'outputs'
# output_dir.mkdir(parents=True, exist_ok=True)
# for name, payload in results.items():
#     payload['metrics'].to_csv(output_dir / f'{name}_link_metrics.csv', index=False)
# print(f'Saved metrics to {output_dir}')
